# Android III

## Permissions
- An application in Android can only use resources that it "owns"
    - Your app owns very few resources to start with
- To protect user privacy, as well as allow for easy management of privacy, the use of any resource not controlled by your application requires a permission

## Normal vs Dangerous
- Android has two levels of permission
    - Normal
    - Dangerous
- Both must be explicitly requested!

## Permissions in the Manifest
- Permissions are requested in the Manifest
    - Directly under the `<manifest>` tag
- `<use-permission>` tag
    - `android:name` Attribute specificies the permission requested
- List of permissions available at https://developer.android.com/reference/android/Manifest.permission.html

## Normal Permissions 
- These resources don't compromise user privacy or data
- Are granted by Android on installation
- Examples:
    - Vibration
    - Gettinng the size of all packages
    - Getting statistics about the battery
    - Using the internet

## Dangerous
- Resources that might perform sensitive operations
     - Sending messages
     - Answering Phone Calls
     - Reading the Contact List
     - Reading external storage
- The user must explicitly grant these permissions
    - In newer versions of Android it is your responsibilty to ask!
  

## Permissions in Android 6+
- Starting with Android M (Marshmallow/ 6.0) permissions are granted slightly differently
- Rather than asking for all needed permissions at installation, they are requested as needed through use of the app
    - Still are all listed in the Manifest, the user is not prompted until they are needed
- The user can revoke permissions at any time through the OS settings

## Checking Permissions at Run TIme
- Because permissions can change between runs of an app, it is not safe to assume they are granted
- Check them ar Runtime!
```java
if(ContextCompat.checkSelfPermission(this,
            Manifest.permission.PERMISSION_CONSTANT) == 
            PackageManager.PERMISSION_GRANTED){
}
```
- If you don't have them you must ask for them
```java
requestPermissions(this, String PermissionsArray[], requestCode)
```

## Responding to a Permissions Request
- When we request permissions, the OS uses a dialog box to prompt for them
    - This places our activity in the paused state, as it is partially obscured
- The `onRequestPermissionsResult` event handler is fired when the activity is resumed
    - Has as parameters the requestCode, the permissions array, and an array of which permissions were granted
    - Should do the same thing you were about to do when you requested the permissions

## Avoiding Permissions
- Using too many permissions makes users suspicious of your app
    - It also is a sign you are doing too much on your own
- Whenever possible, use an intent to launch another activity rather than request permissions to do it yourself

## Storage on Android
- Storage space on Android is divided into two main areas
    - Internal
    - External
        - Name is a reflection of earlier phones that used a physically external device like an SD card
        - Often is one the same disk as Internal storage now
- The permissions needed and the privacy given are different for each

## Using Internal Storage
- Internal storage is private to your app and requires no permission
- There are two methods inherited from Context that let use use internal storage
    - `context.getFilesDir()` returns a java.io.File object representing the location
    - `openFileOutput()` returns a java.io.FileOutputStream object represinting the actualy stream
        - Needs to be passed `Context.MODE_PRIVATE` to open in Internal Storage

## External Storage
- External Storage requires permission using either
    - `WRITE_EXTERNAL_STORAGE`
    - `READ_EXTERNAL_STORAGE`
- This is usually public and readable by all apps and users on the system
    - Sticks around after app is uninstalled
- Might not be available
    - User could mount their phone as USB hardrive on computer

## Using External Storage
- First Check if the extenral storage is available for whatever you are trying to do with it
```java
 String state = Environment.getExternalStorageState();
    if (Environment.MEDIA_MOUNTED[_READ_ONLY].equals(state)) {
       
    }
```
- Next get the appropriate external location
    - Public
    - Private

## Public External Storage
- Meant to be a common location all apps use
    - Pictures
    - Ringtones
```java
Environment.getExternalStoragePublicDirectory(
            Environment.DIRECTORY_CONSTNANT);
```
- A full list of public directories is available at https://developer.android.com/reference/android/os/Environment.html

## Private External Storage
- This is still world-readable and not private in a privacy sense
- It is known as private because of how android structures the external storage directory
    - Associated with your app only
    - Will be deleted when app is uninstalled
```java
getExternalFilesDir(null)
```

## Camera Example
- We will talk through this, it's actually a little complicated 

## Other Storage Mechanisms
- Android has built in support for several other storage mechanisms
    - Network based storage
    - Databases
        - SQLLite locally
        - Firebase over the network
    - Shared Preferences

## Shared Preferences
- Shared Preferences are another type of private data 
- Are used to store small key-value pairs accross runs of the app
    - Things like preferences
- You need to create the file, but normally there is just one per app
```
private String sharedPrefFile = "your.package.namesharedprefs";
mPreferences = getSharedPreferences(sharedPrefFile, MODE_PRIVATE);
```

## Shared Preferences
- The Shared Perferences object doesn't allow you to directly edit it
    - Use the object to get a `SharedPreferences.Editor object`
    - Call the various `put` methods of the editor
    - Call the `apply` method to "commit" the preferences
- This is normally done in `onPause`

## Sensors
- The hardware on Android devices is usually equipped with various sensors that can also cause events
    - A list of Sensors can be found at  https://source.android.com/devices/sensors/sensor-types
- These event must be attached programatically
- The events are registed with a SensorManager object by passing the listener object and the sensor to listen for events on
```java
SensorManager is provided by the OS and retrieved using
mySensorManager = (SensorManager) getSystemService(
    Context.SENSOR_SERVICE);
```

## SensorEventListener
- Just like for input events, the sensor events are handled by classes that implement SensorEventListener
- This provides two methods that need to be overridden
    - `onSensorChanged(SensorEvent event)` which is called when the sensor value changes
    - `onAccuracyChanged(SensorEvent event, int accuracy)` which is called when the accuracy of the sensor changes for some reason

## Biding a Listener to a Sensor
- To attach the listener, an object representing the sensor must be retrieved from the sensor manager object
```java
temp = mSensorManager.getDefaultSensor(Sensor.TYPE_TEMPERATURE);
```
- The listener is then attached using registerListener and passing the listener, the sensor, and the delay between sensor events
```java
mySensorManager.registerListener(new SensorEventListener(), temperature, SensorManager.SENSOR_DELAY_NORMAL);
```

## Using Location 
- When building an app there are two different APIs that can be used for location
    - The `android.location` package
        - Provides low level access to location API
        - Doesn't require Google Play Services, is soley part of the Android system
    - The `com.google.android.gms.location` package
        - Higer level API
        - Requires Google approved device
        - Can be updated seperately from the OS

## Play Services API Location
- To use location from play services, first request permission for location
```xml
<uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION"/>
```
- Then a new client is created with specifications for the required API and the behavior on connection to that API
```java
mGoogleApiClient = LocationServices.getFusedLocationProviderClient()
```
- Use `getLastLocation` method to attach listeners to a task
```
mGoogleApiClient.getLastLocation().addOnSuccessListener(
       new OnSuccessListener<Location>() {
           @Override
           public void onSuccess(Location location) {
           ...
```